# Algorithms for Data Science -- Laboratory 4
Author: Pablo Mollá Chárlez

# Counting Distinct Items

The Flajolet-Martin (FM) algorithm is a $\textcolor{green}{\text{probabilistic algorithm}}$ $\textcolor{green}{\text{used for estimating}}$ $\textcolor{green}{\text{the number of distinct elements}}$ $\textcolor{green}{\text{(cardinality) in a large data stream}}$, especially when storing all the data is impractical due to space limitations. It provides a memory-efficient way to estimate the count of distinct items in streaming data, which makes it ideal for applications like web analytics, network traffic monitoring, and database systems.

- $\textcolor{red}{\text{Why not count the distinct items directly?}}$

    For large datasets, counting distinct elements exactly requires either storing the entire dataset or using a large amount of memory. This is inefficient when the dataset is huge, or when you're dealing with continuous streams of data that you cannot store entirely. Instead, Flajolet-Martin leverages hash functions and bit patterns to estimate the count using much less memory.

## Basic Example: Single Hash Function

Here's a simple example to understand the main idea of the **Flajolet-Martin approach**.

Let's say we have a small **data stream** of 4 distinct items:
$$
\textcolor{green}{Items} \text{: ['apple', 'banana', 'orange', 'apple', 'grape', 'banana']}
$$

Our $\textcolor{orange}{\text{goal is to estimate the number of distinct items}}$, which in this case are $apple$, $banana$, $orange$, and $grape$, so the true distinct count is 4.

### Step 1: $\fbox{Hash the Items}$

We apply a hash function to each item. For simplicity, let's pretend we hash the items to these numbers (in binary):
$$
\text{hash('apple')}  = 101000 ( 40 \text{ in decimal}) \\
\text{hash('banana')} = 1100000 ( 96 \text{ in decimal}) \\
\text{hash('orange')} = 10000 ( 16 \text{ in decimal}) \\
\text{hash('grape')} = 101000 ( 40 \text{ in decimal}) 
$$


### Step 2: $\fbox{Count Trailing Zeros}$

Next, we $\textcolor{orange}{\text{count the number of trailing zeros}}$ (the trailing zeros are a sequence of 0 in the decimal representation of a number, after which no other digits follow) in each binary hash:

- $\text{hash('apple')}$ = 101000 has **3 trailing zeros**.
- $\text{hash('banana')}$ = 1100000 has **5 trailing zeros**.
- $\text{hash('orange')}$ = 10000 has **4 trailing zeros**.
- $\text{hash('grape')}$ = 101000 (same as 'apple') has **3 trailing zeros**.


### Step 3: $\fbox{Track the Maximum Trailing Zeros}$

We $\textcolor{orange}{\text{keep track of the maximum number of trailing zeros}}$ across all hashed values:
- $\textcolor{green}{\text{Maximum number of trailing zeros}}$ = $\textcolor{green}{5}$ (from $'banana'`).

### Step 4: **Estimate the Distinct Count**

The Flajolet-Martin algorithm estimates the number of distinct items as:
$$
\text{Estimate} = 2^{\text{Max Trailing Zeros}} = 2^5 = 32
$$

### Explanation

In this example, the algorithm **overestimates** the number of distinct items because we only had 4 distinct items. The estimate is 32, which shows that with a small number of hash functions or items, the algorithm can be imprecise. But as the stream size grows and with multiple hash functions, the estimate becomes much more accurate.


## Refined Example: Multiple Random Hash Functions

Let's say $\textcolor{green}{\text{we have 5 hash functions}}$, and for each one, the maximum number of trailing zeros (out of 1000 hashed values from the stream) is as follows:

- $h1$ = 10 trailing zeros
- $h2$ = 9 trailing zeros
- $h3$ = 11 trailing zeros
- $h4$ = 12 trailing zeros
- $h5$ = 9 trailing zeros

### $\fbox{1. Average of the Estimators}$
As mentioned, the first estimation using the $\textcolor{green}{\text{average}}$ is:

$$
\text{Average estimation} = \frac{2^{10} + 2^{9} + 2^{11} + 2^{12} + 2^{9}}{5}
= \frac{1024 + 512 + 2048 + 4096 + 512}{5}
= \frac{8192}{5} = 1638.4
$$

### $\fbox{2. Median of the Estimators}$
For the $\textcolor{purple}{\text{median}}$ method, we take the powers of 2 for each trailing zero count and then find the median of those values:

The estimations based on trailing zeros:
- $h1$ = \(2^{10} = 1024\)
- $h2$ = \(2^9 = 512\)
- $h3$ = \(2^{11} = 2048\)
- $h4$ = \(2^{12} = 4096\)
- $h5$ = \(2^9 = 512\)

Now, sort these estimations:
$$
[512, 512, 1024, 2048, 4096]
$$

The $\textcolor{purple}{\text{median}}$ value is the middle one, which is 1024. 

### $\fbox{3. Group median method}$:

For the $\textcolor{orange}{\text{group median method}}$, let's divide the 5 estimators into groups. Since 5 is a small number, we can group them into two groups:
- Group 1: $2^{10} = 1024$, $2^{9} = 512$, $2^{11} = 2048$
- Group 2: $2^{12} = 4096$, $2^{9} = 512$

Now, $\textcolor{orange}{\text{compute the median}}$ of each group:
- Median of Group 1: $[512, 1024, 2048]$ $\Longrightarrow$ **1024**.
- Median of Group 2: $[512, 4096]$ $\Longrightarrow$ $\frac{512+4096}{2} = \frac{4608}{2} = $**2048**.

Finally, we take the $\textcolor{orange}{\text{average** of these two medians}}$:
$$
\text{Group median average} = \frac{1024 + 2048}{2} = 1536
$$

### Summary of Estimates:
- **Average estimation**: $1638.4$
- **Median estimation**: $1024$
- **Group median average**: $1536$


## 1. Preliminaries 

The objective of this lab is to implement the Flajolet-Martin approach to count distinct items. First, we generate an universe of $N$ strings of length $12$, and take $d$ items which will constitute our universe of distinct items.

In [40]:
import random
from string import ascii_lowercase

# Parameters
# Universe of N 
N = 256
# Distinct items
d = 3 
# size of stream
stream_size = 10000

# Generate some random strings of size 10
U = []
for _ in range(N):
  U.append(''.join(random.choice(ascii_lowercase) for i in range(12)))

D = random.sample(U,k=d)

print(D)

['mtfsdkgpdbpf', 'mxiufjqqswlc', 'puydmimtohdz']


## 2. Flajolet-Martin: Creating a Hash Function, Estimating Distinct Items Using Trailing 0s

In the following we create a hash function $h(x)$, which also takes as a parameter a hashable and $N$, and returns a value in the range $\{0, \dots, N-1 \}$. 

We simulate a stream taking random values from $D$, count the trailing $0$s in its hash value, keep the maximum value $R$, and then output $2^R$ as the estimator.

In [41]:
import math
import random
from datetime import datetime

random.seed(datetime.now().timestamp())

def h(x,n):
  return hash(x)%n

# Method for counting trailing 0s
def trailing_0(x):
  x1 = x
  t0 = 0
  while x1%2==0 and x1!=0:
    t0 += 1
    x1 = int(x1/2)
  return t0

# Simulating the stream
R = 0
for _ in range(stream_size):
  # Take a random string from the distinct pool
  s = random.choice(D)
  # Check its hash value
  # To allow more space for hash values
  hv = h(s,2*N)
  r = trailing_0(hv)
  if r>R: R=r

est = int(math.pow(2,R))

print('Estimation of distinct items: %d'%est)

Estimation of distinct items: 2


## 3. **TASK** Flajolet-Martin: Using Multiple Hash Functions

$\textcolor{red}{\text{Implement the refined version of the above estimator}}$, using multiple ($k$) hash functions (use the method of generating several pairs of numbers presented last time in the lab) and compute:

1. The $\textcolor{purple}{\text{average of the k estimators}}$.

2. The $\textcolor{purple}{\text{median of the k estimators}}$.

3. $\textcolor{purple}{\text{Divide the estimators into groups}}$ (vary the group size); $\textcolor{purple}{\text{take the median in each group}}$ and then the $\textcolor{purple}{\text{average over the groups}}$.

$\textcolor{red}{\text{Compare the three method's final outputs}}$. $\textcolor{orange}{\text{What do you notice?}}$

$\fbox{Note}$: You can use the Python 3.4 '_statistics_' package (not available in previous versions) to compute medians, averages, and other statistics.

### 3.1 Parameters & Universe U

In [42]:
# YOUR CODE HERE

import statistics

# Parameters
N = 256  # Universe size
d = 3  # Number of distinct items
stream_size = 10000  # Size of stream
k = 5  # Number of hash functions
group_size = 3  # Group size for group median method
p = 1223543677  # Large prime number for the hash function

# Generating universe U and distinct items D
U = [''.join(random.choice(ascii_lowercase) for i in range(12)) for _ in range(N)]
D = random.sample(U, k=d)

print(f"Universe U ({N}):", U)
print(f"Distinct Items ({d}):", D)

Universe U (256): ['bfeetacddter', 'dvwoyezcufrb', 'gesdghaukxht', 'pgvvpkjckwrl', 'ysfuvoifnjus', 'bdfjbiiwmcej', 'xkbivyxfpavc', 'mdldfsabdmza', 'vshapnitsfcb', 'idjmtrqdqose', 'hiqyskjcoakr', 'ptdehyfscrup', 'cihagvlgbfrw', 'myzagcocdjcb', 'vmhmkghecbjz', 'gcnypvewwaoe', 'rsrdalepycig', 'dzmwpeuyucny', 'pbzlkfrcoqvi', 'npvnbwntajyz', 'yvstffcshyqh', 'syjvtvlirjvd', 'bivlgmmaxydh', 'yghbkaqcbbfe', 'tjblaqkrehxw', 'xyhtbqgxyvfy', 'dboflzoqrzsm', 'dioryfbhpvcb', 'fqxhbsejwoav', 'eoipyiarsfws', 'useuftggymtg', 'dxuspgngxlvl', 'oszpdogfbeiq', 'uctjgiikpzmi', 'xxdskwklzzog', 'hssrefjcelmz', 'rsfhuhsuzjhi', 'itffmwswmdxq', 'jepslevsrdhc', 'piojdoflodhe', 'nhronotacoun', 'batgbmvaxftk', 'nzghbpavumgd', 'dcwbylshvzxd', 'bovahxmiqawl', 'blnjyfprawbf', 'ulusniyiysou', 'bultonhbjwnz', 'njsvwjauuvsh', 'ptgjanlvjxdq', 'xjomirsximqi', 'ingwpfwiuraj', 'qysnksktkdwg', 'hdwknlvrecof', 'yctsxtovznop', 'smlgbrfgkexy', 'vccbpbxwnjco', 'rdribyemlvwk', 'ehhifljkbxlg', 'rbybsgadllok', 'opqpexcxzpuf', 'eehz

### 3.2 Hash Functions with random pairs (a,b)

In [43]:
# Hash function that takes random a, b, p, and universe size n
def h(x, a, b, p, n):
    return ((a * hash(x) + b) % p) % n

### 3.3 Flajolet-Martin Estimator with multiple hash functions

In [44]:
# Flajolet-Martin estimator with multiple hash functions
def flajolet_martin(D, k, stream_size, N, p):
    # Randomly generate k pairs (a, b) for the k hash functions
    hash_functions = [(random.randrange(p), random.randrange(p)) for _ in range(k)]
    
    estimators = []
    
    for a, b in hash_functions:
        R = 0
        for _ in range(stream_size):
            x = random.choice(D)
            hash_value = h(x, a, b, p, 2 * N)
            r = trailing_0(hash_value)
            if r > R:
                R = r
        estimators.append(int(math.pow(2, R)))
    
    return estimators

# Running the Flajolet-Martin algorithm
estimators = flajolet_martin(D, k, stream_size, N, p)

### 3.4 Average of the Estimators

Simply computes the mean of all the estimators from each hash function.

In [45]:
# 1. Average of the estimators
avg_estimator = sum(estimators) / len(estimators)

print("Average of the Estimators:", avg_estimator)

Average of the Estimators: 6.6


### 3.5 Median of the Estimators

Computes the median of the estimators, which can provide a more robust estimate, especially when some estimators are outliers.

In [46]:
# 2. Median of the estimators
median_estimator = statistics.median(estimators)

print("Median of the Estimators:", median_estimator)

Median of the Estimators: 4


### 3.6 Group Median Method

Divides the estimators into groups of size group_size, computes the median within each group, and then averages these medians to get the final estimate.

In [47]:
# 3. Group median method: divide into groups, compute median of each group, and average them
def group_median(estimators, group_size):
    groups = [estimators[i:i + group_size] for i in range(0, len(estimators), group_size)]
    print("Groups:", groups)
    medians = [statistics.median(group) for group in groups]
    print("Medians of each group:", medians)
    return sum(medians) / len(medians)

group_median_estimator = group_median(estimators, group_size)

Groups: [[16, 4, 1], [4, 8]]
Medians of each group: [4, 6.0]


## 4. Results

$\textbf{Comparison of Methods}$:

- The average method can be affected by outliers, as extreme estimators could skew the result.

- The median method tends to give a more robust estimate by ignoring the outliers and focusing on the central tendency of the estimators.

- The group median method provides another level of robustness by taking the median of groups first and then averaging them, which can smooth out the effect of noisy hash functions.

In [48]:
# Results
print("Estimators from each hash function:", estimators)
print(f"Average of estimators: {avg_estimator}")
print(f"Median of estimators: {median_estimator}")
print(f"Group median estimator: {group_median_estimator}")

Estimators from each hash function: [16, 4, 1, 4, 8]
Average of estimators: 6.6
Median of estimators: 4
Group median estimator: 5.0
